In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.integrate import simpson

get_ipython().run_line_magic('matplotlib', 'qt5')

plt.close("all")

resistencia = 50  # 50 ohms o 1Mohm
umbral_deteccion = -0.0035  # Umbral para detectar un pico

# Directorio de trabajo donde se guardará el archivo .npz
directorio_trabajo = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos"

def leer_numero_frames(archivo):
    with open(archivo, 'r') as f:
        for _ in range(6):  # Saltar las primeras 6 líneas
            next(f)
        linea_fastframe = next(f)
    return int(linea_fastframe.split(',')[1])

def procesar_adquisicion(tiempo, voltaje):
    """Detecta picos y realiza la integración basada en el inicio y fin del pico."""
    media_base = np.median(voltaje)  # Se usa la mediana para evitar outliers
    voltaje_sin_base = voltaje - media_base

    indices_pico = np.where(voltaje_sin_base < umbral_deteccion)[0]
    if len(indices_pico) == 0:
        return None

    inicio_pico, fin_pico = indices_pico[0], indices_pico[-1]
    
    tiempo_pico = tiempo[inicio_pico:fin_pico + 1]
    voltaje_pico = voltaje_sin_base[inicio_pico:fin_pico + 1]
    carga = -simpson(voltaje_pico, tiempo_pico) / resistencia

    return carga if carga > -1e-12 else None

def procesar_csv(archivo, tiempo_col='Tiempo (s)', voltaje_col='Voltaje (V)'):
    """Procesa un archivo CSV, detecta picos en cada adquisición y guarda las cargas."""
    N_adquisiciones = leer_numero_frames(archivo)
    datos = pd.read_csv(archivo, skiprows=7)
    tiempo = datos[tiempo_col].values
    voltaje = datos[voltaje_col].values
    
    tamaño_adquisicion = len(tiempo) // N_adquisiciones
    cargas = []

    for i in range(N_adquisiciones):
        inicio, fin = i * tamaño_adquisicion, (i + 1) * tamaño_adquisicion
        carga = procesar_adquisicion(tiempo[inicio:fin], voltaje[inicio:fin])
        if carga is not None:
            cargas.append(carga)
    
    return cargas

def procesar_carpeta(carpeta):
    """Procesa todos los archivos CSV en la carpeta, genera el histograma y guarda todo en un .npz."""
    cargas_totales = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.csv')]

    for archivo in archivos:
        try:
            cargas = procesar_csv(os.path.join(carpeta, archivo))
            cargas_totales.extend(cargas)
        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
    
    cargas_arr = np.array(cargas_totales)
    
    # Filtrar cargas mayores a 0.3e-9 C
    cargas_filtradas = cargas_arr[cargas_arr <= 6.5e-12]
    
    # Generar y mostrar el histograma
    plt.figure(figsize=(12, 6))
    counts, bins, _ = plt.hist(cargas_filtradas, bins=500, edgecolor='black', alpha=0.7, log=True)
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Número de cuentas (escala logarítmica)')
    plt.title(f'Histograma de Cargas - {len(cargas_filtradas)} eventos (filtrado ≤1e-11 C)')
    plt.show()
    
    # Calcular los centros de los bins
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Guardar todo en un archivo .npz en el directorio de trabajo
    datos_path = os.path.join(directorio_trabajo, 'datos_histograma4.npz')
    np.savez(datos_path, 
             cargas_totales=cargas_arr, 
             bin_centers=bin_centers, 
             frecuencias=counts)
    print(f"Todos los datos (cargas totales, centros de bins y frecuencias) guardados en: {datos_path}")
    
    return cargas_totales  # Devuelve todas las cargas (sin filtrar) por si las necesitas

# Ruta de la carpeta con los archivos CSV
carpeta_csv = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k"
cargas = procesar_carpeta(carpeta_csv)

Todos los datos (cargas totales, centros de bins y frecuencias) guardados en: C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos\datos_histograma4.npz


# cargo histo

In [390]:
import numpy as np
import matplotlib.pyplot as plt

datos_path = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos\datos_histograma4.npz"
datos = np.load(datos_path)

cargas_totales = datos['cargas_totales']
bin_centers = datos['bin_centers']
frecuencias = datos['frecuencias']

plt.figure(figsize=(12, 6))
plt.bar(bin_centers, frecuencias, width=np.diff(bin_centers)[0], edgecolor='black', alpha=0.7)
#plt.yscale('log')
plt.xlabel('Carga (Coulombs)')
plt.ylabel('Número de cuentas (escala logarítmica)')
plt.title(f'Histograma de Cargas - {len(cargas_totales[cargas_totales <= 0.3e-9])} eventos')
plt.show()

# ajuste spe perfecto version final pe=0.1

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.interpolate import UnivariateSpline
from scipy.signal import find_peaks

#plt.close("all")
get_ipython().run_line_magic('matplotlib', 'qt5')

# Directorio donde está el archivo .npz
directorio_trabajo = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos"
datos_path = f"{directorio_trabajo}\datos_histograma4.npz"

# Definir la función S_0(x) para el ajuste, ahora con normalización N
def S0(x, N, A, mu, sigma):
    """Función S_0(x) para ajustar al histograma, con un factor de normalización N."""
    PE = 0.083  # Valor fijo
    
    # Calcular g_N (normalización de la parte gaussiana)
    gN = 0.5 * (1 + erf(mu / (np.sqrt(2) * sigma)))
    
    # Término exponencial (normalizado por A)
    term1 = (PE / A) * np.exp(-x / A)
    
    # Término gaussiano (normalizado por gN y sigma)
    term2 = ((1 - PE) / gN) * (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu)**2) / (2 * sigma**2))
    
    return N * (term1 + term2)  # N es el factor de escala global

def ajustar_histograma(datos_path):
    """Carga los datos del .npz, suaviza el histograma como envolvente y ajusta la función S_0(x)."""
    # Cargar los datos
    datos = np.load(datos_path)
    bin_centers = datos['bin_centers']
    frecuencias = datos['frecuencias']
    
    bin_centers = bin_centers[5:]
    frecuencias = frecuencias[5:]

    # Filtrar para x > 0
    mask = bin_centers > -3e-14
    x_data = bin_centers[mask]
    y_data = frecuencias[mask]
    
    # Filtrar datos con frecuencias mayores a 0 para evitar problemas con log
    mask_valid = y_data > 0
    x_data = x_data[mask_valid]
    y_data = y_data[mask_valid]
    
    # Paso 1: Identificar los picos del histograma
    # Usamos find_peaks para encontrar los picos locales
    # Ajustamos el parámetro height para considerar solo picos significativos
    umbral_altura = 0.05 * np.max(y_data)  # Consideramos picos que sean al menos el 5% del máximo
    picos, _ = find_peaks(y_data, height=umbral_altura, distance=5)  # distance evita picos muy cercanos
    
    # Extraer las coordenadas de los picos
    x_picos = x_data[picos]
    y_picos = y_data[picos]
    
    # Paso 2: Ajustar un spline a los picos para crear la envolvente
    # Usamos UnivariateSpline con un parámetro de suavizado pequeño para que pase cerca de los picos
    spline = UnivariateSpline(x_picos, y_picos, s=0.05*len(x_picos), k=3)  # k=3 para un spline cúbico
    x_smooth = np.linspace(min(x_data), max(x_data), 100)
    y_smooth = spline(x_smooth)
    
    # Asegurarnos de que y_smooth no tenga valores negativos
    y_smooth = np.maximum(y_smooth, 0)
    
    # Estimaciones iniciales para los parámetros
    N_init = 500e-12
    A_init = 1.61e-13  # Escala típica de las cargas
    mu_init = 3e-12  # Cerca del pico del histograma
    sigma_init = 3e-12  # Ancho pequeño para la gaussiana
    p0 = [N_init, A_init, mu_init, sigma_init]
    
    # Límites para los parámetros: N > 0, A > 0, mu > 0, sigma > 0
    bounds = ([100e-13, 1.6e-13, 1.62e-12, 1e-13], [1000e-11, 1e-12, 5e-12, 3e-12])
    
    # Ajustar la función S_0(x)
    try:
        popt, pcov = curve_fit(S0, x_smooth, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
        N, A, mu, sigma = popt
        errors = np.sqrt(np.diag(pcov))  # Errores de los parámetros
        
        # Generar la curva ajustada
        y_fit = S0(x_smooth, *popt)
        
        # Calcular chi cuadrado
        residuals = y_smooth - y_fit
        chi_squared = np.sum((residuals)**2 / y_fit)  # Chi cuadrado
        dof = len(x_smooth) - len(popt)  # Grados de libertad
        reduced_chi_squared = chi_squared / dof  # Chi cuadrado reducido
        
        # Graficar el histograma real, la envolvente y el ajuste
        plt.figure(figsize=(12, 6))
        plt.bar(x_data, y_data, width=np.diff(x_data[0:2])[0], alpha=0.5, color='blue', label='Histograma')  # Histograma real
        plt.plot(x_smooth, y_fit, 'r-', label=(
            f'Ajuste S_0(x)\n'
            f'A = ({A:.2e} ± {errors[1]:.2e})\n'
            f'μ = ({mu:.2e} ± {errors[2]:.2e})\n'
            f'σ = ({sigma:.2e} ± {errors[3]:.2e})\n'
            f'χ²/ν = {reduced_chi_squared:.2f}'
        ))
        plt.xlabel('Carga (Coulombs)')
        plt.ylabel('Número de cuentas')
        plt.title(r'$S_0(x) = \frac{PE}{A} e^{-x/A} + \frac{1-PE}{g_N} \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$')
        plt.legend()
        plt.show()
        
        print("Parámetros ajustados con errores:")
        print(f"N (normalización): {N:.2e} ± {errors[0]:.2e}")
        print(f"A (exponencial): {A:.2e} ± {errors[1]:.2e}")
        print(f"μ (gaussiana): {mu:.2e} ± {errors[2]:.2e}")
        print(f"σ (gaussiana): {sigma:.2e} ± {errors[3]:.2e}")
        print(f"\nChi cuadrado reducido: {reduced_chi_squared:.2f}")
        
        # Calcular el área bajo la curva ajustada y comparar con los datos
        area_datos = np.trapz(y_smooth, x_smooth)
        area_ajuste = np.trapz(y_fit, x_smooth)
        print(f"\nÁrea bajo la envolvente: {area_datos:.2e}")
        print(f"Área bajo el ajuste: {area_ajuste:.2e}")
        print(f"Relación áreas (ajuste/envolvente): {area_ajuste / area_datos:.2f}")
        
    except RuntimeError as e:
        print(f"No se pudo realizar el ajuste: {e}")

# Ajustar la función S_0(x) a la envolvente del histograma
ajustar_histograma(datos_path)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_16132\3857237045.py:13: SyntaxWarning: invalid escape sequence '\d'
  datos_path = f"{directorio_trabajo}\datos_histograma4.npz"


Parámetros ajustados con errores:
N (normalización): 1.94e-10 ± 3.81e-12
A (exponencial): 1.60e-13 ± 7.06e-14
μ (gaussiana): 1.62e-12 ± 6.95e-14
σ (gaussiana): 1.60e-12 ± 7.26e-15

Chi cuadrado reducido: 1.26

Área bajo la envolvente: 1.78e-10
Área bajo el ajuste: 1.86e-10
Relación áreas (ajuste/envolvente): 1.04


C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_16132\3857237045.py:119: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area_datos = np.trapz(y_smooth, x_smooth)
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_16132\3857237045.py:120: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area_ajuste = np.trapz(y_fit, x_smooth)


# ESTE ajuste SPE con pe ajustado


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.interpolate import UnivariateSpline
from scipy.signal import find_peaks

#plt.close("all")
get_ipython().run_line_magic('matplotlib', 'qt5')

# Directorio donde está el archivo .npz
directorio_trabajo = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos"
datos_path = f"{directorio_trabajo}\datos_histograma4.npz"

# Definir la función S_0(x) para el ajuste, ahora con PE como parámetro ajustable
def S0(x, N, PE, A, mu, sigma):
    """Función S_0(x) para ajustar al histograma, con PE (probabilidad) como parámetro."""
    # Validar que PE esté entre 0 y 1
    PE = np.clip(PE, 0, 1)
    
    # Calcular g_N (normalización de la parte gaussiana)
    gN = 0.5 * (1 + erf(mu / (np.sqrt(2) * sigma)))
    
    # Término exponencial (normalizado por A)
    term1 = (PE / A) * np.exp(-x / A)
    
    # Término gaussiano (normalizado por gN y sigma)
    term2 = ((1 - PE) / gN) * (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu)**2) / (2 * sigma**2))
    
    return N * (term1 + term2)  # N es el factor de escala global

def ajustar_histograma(datos_path):
    """Carga los datos del .npz, suaviza el histograma como envolvente y ajusta la función S_0(x)."""
    # Cargar los datos
    datos = np.load(datos_path)
    bin_centers = datos['bin_centers']
    frecuencias = datos['frecuencias']
    
    bin_centers = bin_centers[5:]
    frecuencias = frecuencias[5:]

    # Filtrar para x > 0
    mask = bin_centers > -3e-14
    x_data = bin_centers[mask]
    y_data = frecuencias[mask]
    
    # Filtrar datos con frecuencias mayores a 0 para evitar problemas con log
    mask_valid = y_data > 0
    x_data = x_data[mask_valid]
    y_data = y_data[mask_valid]
    
    # Paso 1: Identificar los picos del histograma
    umbral_altura = 0.05 * np.max(y_data)
    picos, propiedades = find_peaks(y_data, height=umbral_altura, distance=6)
    
    # Extraer las coordenadas de los picos
    x_picos = x_data[picos]
    y_picos = y_data[picos]
    
    # Paso 2: Ajustar un spline a los picos para crear la envolvente
    spline = UnivariateSpline(x_picos, y_picos, s=0.05*len(x_picos), k=3)
    x_smooth = np.linspace(min(x_data), max(x_data), 100)
    y_smooth = spline(x_smooth)
    y_smooth = np.maximum(y_smooth, 0)
    
    # Estimaciones iniciales para los parámetros (incluyendo PE)
    N_init = 500e-12
    PE_init = 0.5  # Valor inicial de PE
    A_init = 1.6e-13
    mu_init = 3e-12
    sigma_init = 3e-12
    p0 = [N_init, PE_init, A_init, mu_init, sigma_init]
    
    # Límites para los parámetros: 
    # N > 0, 0 ≤ PE ≤ 1, A > 0, mu > 0, sigma > 0
    bounds = (
        [100e-13, 0, 1.5e-13, 1.724e-12, 1e-13],  # Límites inferiores
        [1000e-11, 1, 2e-13, 5e-12, 3e-12]       # Límites superiores
    )
    
    # Ajustar la función S_0(x)
    try:
        popt, pcov = curve_fit(S0, x_smooth, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
        N, PE, A, mu, sigma = popt
        errors = np.sqrt(np.diag(pcov))
        
        # Generar la curva ajustada
        y_fit = S0(x_smooth, *popt)
        
        # Calcular chi cuadrado
        residuals = y_smooth - y_fit
        chi_squared = np.sum((residuals)**2 / y_fit)
        dof = len(x_smooth) - len(popt)
        reduced_chi_squared = chi_squared / dof
        
        # Graficar el histograma real, los picos, la envolvente y el ajuste
        plt.figure(figsize=(12, 6))
        plt.bar(x_data, y_data, width=np.diff(x_data[0:2])[0], alpha=0.5, color='blue', label='Histograma')
        plt.scatter(x_picos, y_picos, color='green', s=100, label='Picos detectados', zorder=5)  # Picos
        plt.plot(x_smooth, y_smooth, 'k--', label='Envolvente (spline)', linewidth=2)  # Envolvente
        plt.plot(x_smooth, y_fit, 'r-', label=(
            f'Ajuste S_0(x)\n'
            f'PE = {PE:.3f} ± {errors[1]:.3f}\n'
            f'A = {A:.3e} ± {errors[2]:.3e}\n'
            f'μ = {mu:.3e} ± {errors[3]:.3e}\n'
            f'σ = {sigma:.3e} ± {errors[4]:.3e}\n'
            f'χ²/ν = {reduced_chi_squared:.3f}'
        ), linewidth=2)
        plt.xlabel('Carga (Coulombs)')
        plt.ylabel('Número de cuentas')
        plt.title(r'$S_0(x) = \frac{PE}{A} e^{-x/A} + \frac{1-PE}{g_N} \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        print("Parámetros ajustados con errores:")
        print(f"N (normalización): {N:.3e} ± {errors[0]:.3e}")
        print(f"PE (probabilidad): {PE:.3f} ± {errors[1]:.3f}")
        print(f"A (exponencial): {A:.3e} ± {errors[2]:.3e}")
        print(f"μ (gaussiana): {mu:.3e} ± {errors[3]:.3e}")
        print(f"σ (gaussiana): {sigma:.3e} ± {errors[4]:.3e}")
        print(f"\nChi cuadrado reducido: {reduced_chi_squared:.2f}")
        
    except RuntimeError as e:
        print(f"No se pudo realizar el ajuste: {e}")

# Ajustar la función S_0(x) a la envolvente del histograma
ajustar_histograma(datos_path)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_14048\951729577.py:13: SyntaxWarning: invalid escape sequence '\d'
  datos_path = f"{directorio_trabajo}\datos_histograma4.npz"


Parámetros ajustados con errores:
N (normalización): 1.943e-10 ± 3.179e-12
PE (probabilidad): 0.124 ± 0.005
A (exponencial): 2.000e-13 ± 1.287e-14
μ (gaussiana): 1.724e-12 ± 5.791e-14
σ (gaussiana): 1.551e-12 ± 6.286e-15

Chi cuadrado reducido: 1.06


# visualizacion func spe

In [710]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf

def S0(x, N, A, mu, sigma):
    """Función S_0(x) para ajustar al histograma, con un factor de normalización N."""
    PE = 0.917  # Valor fijo
    
    # Calcular g_N (normalización de la parte gaussiana)
    gN = 0.5 * (1 + erf(mu / (np.sqrt(2) * sigma)))
    
    # Término exponencial (normalizado por A)
    term1 = (PE / A) * np.exp(-x / A)
    
    # Término gaussiano (normalizado por gN y sigma)
    term2 = ((1 - PE) / gN) * (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu)**2) / (2 * sigma**2))
    
    return N * (term1 + term2)

# Parámetros ajustados para mayor participación gaussiana
N = 1000    # Factor de normalización
A = 2       # Aumentamos A para reducir la contribución exponencial
mu = 7.5      # Centro de la gaussiana
sigma = 1   # Aumentamos sigma para una gaussiana más ancha y prominente

# Crear datos para graficar
x = np.linspace(0, 20, 500)  # Ampliamos el rango para ver la gaussiana más ancha
y = S0(x, N, A, mu, sigma)

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'b-', label=f'S0(x) (PE={0.917}, N={N}, A={A}, μ={mu}, σ={sigma})')
plt.grid(True)
plt.xlabel('x')
plt.ylabel('S0(x)')
plt.title('Visualización de S0(x) con mayor participación gaussiana')
plt.legend()
plt.show()

# ajuste gral bien

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.interpolate import UnivariateSpline
from scipy.signal import find_peaks
from scipy.special import factorial

# Definir la función f(x)
def f(x, N, lambda_, A, mu, sigma, mu_ped, sigma_ped):
    PE = 0.917  # Valor fijo
    gN = 0.5 * (1 + erf(mu / (np.sqrt(2) * sigma)))
    
    term1 = (PE / (2 * A)) * np.exp((sigma_ped**2 - A * (x - mu_ped)) / (A**2)) * \
            (1 + erf((A * (x - mu_ped) - sigma_ped**2) / (np.sqrt(2) * A * sigma_ped)))
    
    term2 = ((1 - PE) / gN) * (1 / np.sqrt(2 * np.pi * (sigma**2 + sigma_ped**2))) * \
            np.exp(-((x - mu_ped)**2) / (2 * (sigma**2 + sigma_ped**2)))
    
    term3 = (1 / np.sqrt(2 * np.pi * sigma_ped**2)) * np.exp(-((x - mu_ped)**2) / (2 * sigma_ped**2))
    
    term4 = 0
    for n in range(2, 5):
        poisson = np.exp(-lambda_) * (lambda_**n) / factorial(n)
        gauss = (1 / np.sqrt(2 * np.pi * (n * sigma**2 + sigma_ped**2))) * \
                np.exp(-((x - n * mu - mu_ped)**2) / (2 * (n * sigma**2 + sigma_ped**2)))
        term4 += poisson * gauss
    
    result = np.exp(-lambda_ * x) * (term1 + term2) + term3 + term4
    return N * result

# Función para ajustar el histograma usando la envolvente spline
def ajustar_histograma(datos_path):
    datos = np.load(datos_path)
    bin_centers = datos['bin_centers']
    frecuencias = datos['frecuencias']
    
    bin_centers = bin_centers[5:]
    frecuencias = frecuencias[5:]

    mask = bin_centers > -3e-14
    x_data = bin_centers[mask]
    y_data = frecuencias[mask]
    
    mask_valid = y_data > 0
    x_data = x_data[mask_valid]
    y_data = y_data[mask_valid]
    
    picos, _ = find_peaks(y_data, height=0.05 * np.max(y_data), distance=5)
    x_picos = x_data[picos]
    y_picos = y_data[picos]
    
    spline = UnivariateSpline(x_picos, y_picos, s=0.05 * len(x_picos), k=3)
    x_smooth = np.linspace(min(x_data), max(x_data), 100)
    y_smooth = spline(x_smooth)
    y_smooth = np.maximum(y_smooth, 0)
    #["N", "lambda_", "A", "mu", "sigma", "mu_ped", "sigma_ped"]
    p0 = [500e-18, 1.0, 1.61e-13, 3e-12, 3e-12, 0, 1e-12]
    bounds = ([100e-200, 1e-3, 1.6e-13, 1e-12, 1e-12, -1e-12, 1e-13],
              [1000e-14,1e3 , 1e-12, 5e-12, 3e-12, 1e-12, 3e-12])
    
    try:
        popt, pcov = curve_fit(f, x_smooth, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
        y_fit = f(x_smooth, *popt)
        residuals = y_smooth - y_fit
        chi_squared = np.sum((residuals)**2 / y_fit)
        dof = len(x_smooth) - len(popt)
        reduced_chi_squared = chi_squared / dof
        
        # Mostrar parámetros ajustados en consola
        print("Parámetros ajustados:")
        param_names = ["N", "lambda_", "A", "mu", "sigma", "mu_ped", "sigma_ped"]
        for name, value in zip(param_names, popt):
            print(f"{name} = {value:.3e}")
        print(f"Chi-cuadrado reducido: {reduced_chi_squared:.2f}")
        
        # Mostrar parámetros en el gráfico
        param_text = "\n".join([f"{name} = {value:.3e}" for name, value in zip(param_names, popt)])
        param_text += f"\n$\chi^2/\nu$ = {reduced_chi_squared:.2f}"
        
        plt.figure(figsize=(12, 6))
        plt.bar(x_data, y_data, width=np.diff(x_data[0:2])[0], alpha=0.5, color='blue', label='Histograma')
        plt.plot(x_smooth, y_fit, 'r-', label='Ajuste f(x)')
        plt.xlabel('Carga (Coulombs)')
        plt.ylabel('Número de cuentas')
        plt.legend()
        plt.text(0.05, 0.95, param_text, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))
        plt.show()
    except RuntimeError as e:
        print(f"No se pudo realizar el ajuste: {e}")

# Ruta del archivo npz
datos_path = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\codigos/datos_histograma4.npz"
ajustar_histograma(datos_path)


<>:79: SyntaxWarning: invalid escape sequence '\c'
<>:79: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_5776\448408111.py:79: SyntaxWarning: invalid escape sequence '\c'
  param_text += f"\n$\chi^2/\nu$ = {reduced_chi_squared:.2f}"


Parámetros ajustados:
N = 9.634e-12
lambda_ = 1.972e+00
A = 5.221e-13
mu = 1.169e-12
sigma = 2.995e-12
mu_ped = 9.204e-13
sigma_ped = 1.926e-13
Chi-cuadrado reducido: 1152.47
